# C2Cシェアサイクルシステム

## 問題設定
個人間で自転車をシェアリングし，可能な限り乗り捨て可能とするシステムについて考える．利用可能な自転車は複数台存在し，それぞれが任意のポイントに駐輪されているとする．本システムのユーザーのサービス利用リクエストを時間幅Tの間隔で分割し，それぞれのリクエストに合わせた最適な自転車を割り当てる．

サービス利用後に任意の場所に駐輪された自転車を定位置に再配置するコストを最小化し，　より多くのユーザーに自転車を割り当てることを目的として，最適化な自転車をユーザーに割り当てる．

なお，ここではMVPとして，時間幅T=1(分)とした小規模のデータを利用することとし，最終アウトプットとしては「どのユーザーにどの自転車が割り当てられたか」と「移動後の自転車の配置状況・利用可能状況」を期待する．

[fig]

### データ
データとしては，以下のような情報を必要とする．
- 定数
 - $T$：ユーザーリクエストを区切る時間幅
- 変数
 - $J$：ユーザーリクエストデータ(データフレーム？)
 - $B$：利用可能な自転車データ(データフレーム？)

 etc

In [ ]:
# ライブラリのインストール
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.28.1 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.

In [ ]:
import branca.colormap as cm
import folium
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from geopy.distance import geodesic
from ortools.linear_solver import pywraplp
from pandas import DataFrame

In [ ]:
'''ユーザーリクエストJに対して移動後の自転車Bの位置関係を表す距離行列を返す関数'''
def generate_after_trip_distances(
    df_bikes: DataFrame,
    df_requests: DataFrame,
) -> np.ndarray:

    # 自転車とリクエストの数
    num_bikes = len(df_bikes)
    num_requests = len(df_requests)

    # 移動後の距離行列 d を作成 (d[b, j] が利用者 j が移動した後の自転車 b とその定位置までの距離)
    # 距離行列を初期化
    after_trip_distances = np.zeros((num_bikes, num_requests))
    for b in range(num_bikes):
        home_position = df_bikes.loc[b, 'Home Position']
        for j in range(num_requests):
            request_destination_id = df_requests.loc[j, 'DOLocationID']
            request_destination = get_coordinates_by_location_id(request_destination_id)
            after_trip_distances[b, j] = geodesic(
                home_position, request_destination
            ).m  # 単位はメートル

    return after_trip_distances


'''ユーザーリクエストJに対して移動前の自転車Bの位置関係を表す距離行列を返す関数'''
def generate_before_trip_distances(
    df_bikes: pd.DataFrame,
    df_requests: pd.DataFrame,
) -> np.ndarray:

    # 自転車とリクエストの数
    num_bikes = len(df_bikes)
    num_requests = len(df_requests)

    # 移動前の距離行列 d を作成 (d[b, j] が利用者 j のリクエスト地点と自転車 b の現在地との距離)
    # 距離行列を初期化
    before_trip_distances = np.zeros((num_bikes, num_requests))
    for b in range(num_bikes):
        current_location = df_bikes.loc[b, 'Current Location']
        for j in range(num_requests):
            request_pickup_id = df_requests.loc[j, 'PULocationID']
            request_pickup = get_coordinates_by_location_id(request_pickup_id)
            before_trip_distances[b, j] = geodesic(
                current_location, request_pickup
            ).m  # 単位はメートル

    return before_trip_distances


'''利用可能な自転車の集合を返す関数'''
def get_available_bikes(
    df_bikes: pd.DataFrame
) -> np.ndarray:
    # 現在時刻を取得
    # current_time = datetime.now()
    # テストデータとしてcurrent_timeを自作する
    current_time = datetime(2023, 1, 1, 0, 0, 2)

    # 利用可能な自転車を1、不可能な自転車を0とする行列を作成
    available_bikes = (B['DODatetime'].isna() | (B['DODatetime'] < current_time)).astype(int)

    return available_bikes.values



---



# 疑似データ

In [ ]:
import pandas as pd
from datetime import datetime

# 自転車のデータを手動で作成
def generate_bike_data():
    # 自転車のデータを手動で設定
    bike_data = [
        {'Bike ID': 0, 'Home Position': (40.75302540139779, -73.96722485932337), 'Current Location': (40.75537502442001, -73.97718971701514), 'DODatetime': pd.NaT},
        {'Bike ID': 1, 'Home Position': (40.74630172960795, -73.99076945733133), 'Current Location': (40.75209495412983, -73.975436356298), 'DODatetime': pd.NaT},
        {'Bike ID': 2, 'Home Position': (40.75046184888543, -73.97346615591313), 'Current Location': (40.75046184888543, -73.97346615591313), 'DODatetime': pd.NaT},
        {'Bike ID': 3, 'Home Position': (40.754672378913824, -73.98080231506587), 'Current Location': (40.754672378913824, -73.98080231506587), 'DODatetime': pd.NaT},
        {'Bike ID': 4, 'Home Position': (40.75701407362949, -73.97694085905479), 'Current Location': (40.75701407362949, -73.97694085905479), 'DODatetime': pd.NaT}
    ]

    # データフレームに変換
    df_bikes = pd.DataFrame(bike_data)
    df_bikes.set_index('Bike ID', inplace=True)

    return df_bikes

# ユーザーリクエストのデータを手動で作成
def generate_user_requests():
    # ユーザーリクエストのデータを手動で設定
    request_data = [
        {'tpep_pickup_datetime': datetime(2023, 1, 1, 0, 0, 4), 'tpep_dropoff_datetime': datetime(2023, 1, 1, 0, 24, 4), 'PULocationID': '40.756840428905974,-73.97876035333631', 'DOLocationID': '40.76029373426692,-73.9699974138823'},
        {'tpep_pickup_datetime': datetime(2023, 1, 1, 0, 0, 52), 'tpep_dropoff_datetime': datetime(2023, 1, 1, 0, 15, 52), 'PULocationID': '40.75510063317113, -73.97470612575748', 'DOLocationID': '40.757096485326855,-73.98440920093248'},
        {'tpep_pickup_datetime': datetime(2023, 1, 1, 0, 0, 36), 'tpep_dropoff_datetime': datetime(2023, 1, 1, 0, 17, 36), 'PULocationID': '40.75370743826586,-73.97852812156756', 'DOLocationID': '40.75266371250171,-73.97499968784228'},
        # その他のリクエストデータも同様に設定する
    ]

    # データフレームに変換
    df_requests = pd.DataFrame(request_data)
    return df_requests

# マンハッタン内の疑似データの生成
# 自転車データの生成
df_bikes = generate_bike_data()

# ユーザーリクエストデータの生成
df_requests = generate_user_requests()

# 結果の確認
print("Bikes Data (Manhattan):")
print(df_bikes)
print("\nUser Requests (Manhattan):")
print(df_requests)


Bikes Data (Manhattan):
                                    Home Position  \
Bike ID                                             
0         (40.75302540139779, -73.96722485932337)   
1         (40.74630172960795, -73.99076945733133)   
2         (40.75046184888543, -73.97346615591313)   
3        (40.754672378913824, -73.98080231506587)   
4         (40.75701407362949, -73.97694085905479)   

                                 Current Location DODatetime  
Bike ID                                                       
0         (40.75537502442001, -73.97718971701514)        NaT  
1           (40.75209495412983, -73.975436356298)        NaT  
2         (40.75046184888543, -73.97346615591313)        NaT  
3        (40.754672378913824, -73.98080231506587)        NaT  
4         (40.75701407362949, -73.97694085905479)        NaT  

User Requests (Manhattan):
  tpep_pickup_datetime tpep_dropoff_datetime  \
0  2023-01-01 00:00:04   2023-01-01 00:24:04   
1  2023-01-01 00:00:52   2023-01-01 00:

In [ ]:
df_bikes

,Home Position,Current Location,DODatetime
Bike ID,,,
0,"(40.75302540139779, -73.96722485932337)","(40.75537502442001, -73.97718971701514)",NaT
1,"(40.74630172960795, -73.99076945733133)","(40.75209495412983, -73.975436356298)",NaT
2,"(40.75046184888543, -73.97346615591313)","(40.75046184888543, -73.97346615591313)",NaT
3,"(40.754672378913824, -73.98080231506587)","(40.754672378913824, -73.98080231506587)",NaT
4,"(40.75701407362949, -73.97694085905479)","(40.75701407362949, -73.97694085905479)",NaT


In [ ]:
df_requests

,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID
0,2023-01-01 00:00:04,2023-01-01 00:24:04,"40.756840428905974,-73.97876035333631","40.76029373426692,-73.9699974138823"
1,2023-01-01 00:00:52,2023-01-01 00:15:52,"40.75510063317113, -73.97470612575748","40.757096485326855,-73.98440920093248"
2,2023-01-01 00:00:36,2023-01-01 00:17:36,"40.75370743826586,-73.97852812156756","40.75266371250171,-73.97499968784228"


In [ ]:
'''ユーザーリクエストJに対して移動後の自転車Bの位置関係を表す距離行列を返す関数'''
def generate_after_trip_distances(
    df_bikes: DataFrame,
    df_requests: DataFrame,
) -> np.ndarray:

    # 自転車とリクエストの数
    num_bikes = len(df_bikes)
    num_requests = len(df_requests)

    # 移動後の距離行列 d を作成 (d[b, j] が利用者 j が移動した後の自転車 b とその定位置までの距離)
    # 距離行列を初期化
    after_trip_distances = np.zeros((num_bikes, num_requests))
    for b in range(num_bikes):
        home_position = df_bikes.loc[b, 'Home Position']
        for j in range(num_requests):
            request_destination = df_requests.loc[j, 'DOLocationID']
            after_trip_distances[b, j] = geodesic(
                home_position, request_destination
            ).m  # 単位はメートル

    return after_trip_distances


'''ユーザーリクエストJに対して移動前の自転車Bの位置関係を表す距離行列を返す関数'''
def generate_before_trip_distances(
    df_bikes: pd.DataFrame,
    df_requests: pd.DataFrame,
) -> np.ndarray:

    # 自転車とリクエストの数
    num_bikes = len(df_bikes)
    num_requests = len(df_requests)

    # 移動前の距離行列 d を作成 (d[b, j] が利用者 j のリクエスト地点と自転車 b の現在地との距離)
    # 距離行列を初期化
    before_trip_distances = np.zeros((num_bikes, num_requests))
    for b in range(num_bikes):
        current_location = df_bikes.loc[b, 'Current Location']
        for j in range(num_requests):
            request_pickup = df_requests.loc[j, 'PULocationID']
            before_trip_distances[b, j] = geodesic(
                current_location, request_pickup
            ).m  # 単位はメートル

    return before_trip_distances


'''利用可能な自転車の集合を返す関数'''
def get_available_bikes(
    df_bikes: pd.DataFrame
) -> np.ndarray:
    # 現在時刻を取得
    # current_time = datetime.now()
    # テストデータとしてcurrent_timeを自作する
    current_time = datetime(2023, 1, 1, 0, 0, 2)

    # 利用可能な自転車を1、不可能な自転車を0とする行列を作成
    available_bikes = (B['DODatetime'].isna() | (B['DODatetime'] < current_time)).astype(int)

    return available_bikes.values

In [ ]:
import folium
import numpy as np

'''ユーザーの位置と自転車の位置をプロットする関数'''
def plot_users_and_bikes(
    user_locations: np.ndarray,  # ユーザーの位置（ピックアップ地点）
    dropoff_locations: np.ndarray,  # ユーザーの位置（ドロップオフ地点）
    bike_locations: np.ndarray,  # 自転車の現在位置
    home_locations: np.ndarray,  # 自転車のホーム位置
    latitude_range: tuple[float, float],  # 描画範囲 (緯度)
    longitude_range: tuple[float, float],  # 描画範囲 (経度)
):
    tiles = "Cartodb Positron"
    # マップの初期化
    m = folium.Map(
        [sum(latitude_range) / 2, sum(longitude_range) / 2],
        tiles=tiles,
        zoom_start=11,
    )

    # ユーザーの位置（ピックアップ地点）をプロット
    for i, (latitude, longitude) in enumerate(user_locations):
        folium.Marker(
            location=(latitude, longitude),
            icon=folium.Icon(icon="user", prefix="fa", color="orange"),
            tooltip=f"Pickup Location {i}: {latitude:.6f}, {longitude:.6f}"
        ).add_to(m)

        # ピックアップ地点に円を描画
        folium.Circle(
            location=(latitude, longitude),
            radius=250,
            color='orange',
            fill=True,
            fill_opacity=0.2,
            tooltip=f"Pickup Location: {latitude:.6f}, {longitude:.6f}"
        ).add_to(m)

    # 自転車の現在位置とホーム位置をプロットし、点線で結ぶ
    for i, ((bike_lat, bike_lon), (home_lat, home_lon)) in enumerate(zip(bike_locations, home_locations)):
        # 自転車の現在位置をプロット
        folium.Marker(
            location=(bike_lat, bike_lon),
            icon=folium.Icon(icon="bicycle", prefix="fa", color="green"),
            tooltip=f"Bike {i} Current Location: {bike_lat:.6f}, {bike_lon:.6f}"
        ).add_to(m)

        # 自転車のホーム位置をプロット
        folium.Marker(
            location=(home_lat, home_lon),
            icon=folium.Icon(icon="home", prefix="fa", color="green"),
            tooltip=f"Bike {i} Home Location: {home_lat:.6f}, {home_lon:.6f}"
        ).add_to(m)

        # 現在位置とホーム位置を点線で結ぶ
        folium.PolyLine(
            locations=[(home_lat, home_lon), (bike_lat, bike_lon)],
            color='gray',
            weight=1.5,
            opacity=0.5,
            dash_array='5, 10',  # 点線を描画
            tooltip=f"Path from Home to Current: Bike {i}"
        ).add_to(m)

    # ピックアップ地点からドロップオフ地点までの黒い矢印を描画
    for (pu_lat, pu_lon), (do_lat, do_lon) in zip(user_locations, dropoff_locations):
        folium.PolyLine(
            locations=[(pu_lat, pu_lon), (do_lat, do_lon)],
            color='black',
            weight=2.5,
            opacity=0.7,
            tooltip=f"Route from ({pu_lat:.6f}, {pu_lon:.6f}) to ({do_lat:.6f}, {do_lon:.6f})"
        ).add_to(m)

    return m


# # 地図の描画
# m = plot_users_and_bikes(user_locations, bike_locations, (40.74, 40.77), (-73.99, -73.96))
# m.save('map_with_arrows.html')
# print("Map with arrows saved as 'map_with_arrows.html'.")


In [ ]:
# マンハッタンの一部の緯度と経度の範囲
latitude_range = (40.74602504596096, 40.760649290750465)
longitude_range = (-73.99101091076052, -73.9663521077821)
# latitudeカラムとlongitudeカラムの最大値と最小値を取得
latitude_max = df_bikes['Current Location'].apply(lambda x: x[0]).max()
latitude_min = df_bikes['Current Location'].apply(lambda x: x[0]).min()
longitude_max = df_bikes['Current Location'].apply(lambda x: x[1]).max()
longitude_min = df_bikes['Current Location'].apply(lambda x: x[1]).min()

# 結果を表示
print(f"Latitude: max = {latitude_max}, min = {latitude_min}")
print(f"Longitude: max = {longitude_max}, min = {longitude_min}")

Latitude: max = 40.75701407362949, min = 40.75046184888543
Longitude: max = -73.97346615591313, min = -73.98080231506587


In [ ]:
# デバッグ
B = df_bikes
J = df_requests

# ユーザーリクエストJに対して移動された自転車Bにおける、自転車の定位置との距離行列
distances = generate_after_trip_distances(B, J)
print(distances)

initial_distances = generate_before_trip_distances(B, J)
print(initial_distances)

available_bikes = get_available_bikes(B)
print(available_bikes)

[[ 840.41286888 1519.93846714  657.80141572]
 [2343.36752557 1313.58272031 1507.5912784 ]
 [1130.43309414 1181.86953586  276.69424724]
 [1105.50958041  406.49319278  538.3993802 ]
 [ 690.22334397  630.71824626  510.15752775]]
[[209.93538167 211.93045285 216.95039494]
 [597.07760829 339.4267994  316.59913143]
 [837.62913834 525.66812925 559.14455938]
 [296.13977822 516.99141794 219.92068499]
 [154.84982963 284.18695815 390.89791809]]
[1 1 1 1 1]


In [ ]:
# NYC
latitude_range = (latitude_min - 0.1, latitude_max + 0.1)
longitude_range = (longitude_min - 0.1, longitude_max + 0.1)
print(latitude_range)
print(longitude_range)

# 自転車の現在地の配列
current_locations = df_bikes['Current Location'].values
print(type(current_locations))
print(current_locations)

# 自転車のホーム位置の配列
home_locations = df_bikes['Home Position'].values
print(type(home_locations))
print(home_locations)

# ユーザーの現在地を整形する
request_origins = df_requests['PULocationID'].apply(lambda x: tuple(map(float, x.split(',')))).tolist()
request_origins = np.array(request_origins)

print(type(request_origins))
print(request_origins)

# ユーザーのドロップオフ地点の配列
request_destinations = df_requests['DOLocationID'].apply(lambda x: tuple(map(float, x.split(',')))).tolist()
request_destinations = np.array(request_destinations)

print(type(request_destinations))
print(request_destinations)

(40.65046184888543, 40.85701407362949)
(-74.08080231506587, -73.87346615591314)
<class 'numpy.ndarray'>
[(40.75537502442001, -73.97718971701514)
 (40.75209495412983, -73.975436356298)
 (40.75046184888543, -73.97346615591313)
 (40.754672378913824, -73.98080231506587)
 (40.75701407362949, -73.97694085905479)]
<class 'numpy.ndarray'>
[(40.75302540139779, -73.96722485932337)
 (40.74630172960795, -73.99076945733133)
 (40.75046184888543, -73.97346615591313)
 (40.754672378913824, -73.98080231506587)
 (40.75701407362949, -73.97694085905479)]
<class 'numpy.ndarray'>
[[ 40.75684043 -73.97876035]
 [ 40.75510063 -73.97470613]
 [ 40.75370744 -73.97852812]]
<class 'numpy.ndarray'>
[[ 40.76029373 -73.96999741]
 [ 40.75709649 -73.9844092 ]
 [ 40.75266371 -73.97499969]]


In [ ]:
plot_users_and_bikes(request_origins, request_destinations, current_locations, home_locations, latitude_range, longitude_range)

In [ ]:
# 問題の正規化
average = distances.mean()
print(f"average: {average}")
std = distances.std()
print(f"std: {std}")
distances: np.ndarray = (distances - average) / std
print(f"distances: {distances}")

average: 976.8794949745496
std: 528.3622121174127
distances: [[-0.25828234  1.02781569 -0.60390026]
 [ 2.58627131  0.63725834  1.0044469 ]
 [ 0.29062184  0.38797256 -1.32519933]
 [ 0.24345058 -1.07953652 -0.82988545]
 [-0.54253719 -0.65515898 -0.88333714]]


In [ ]:
# OR-Toolsのソルバーを作成
solver = pywraplp.Solver.CreateSolver('SCIP')

# 変数の定義
x = []
for b in range(B.shape[0]):
    x.append([])
    for j in range(J.shape[0]):
        x[b].append(solver.BoolVar(f'x[{b},{j}]'))
print(x)

alpha = 1.0

# 目的関数の定義
# 第一項: ユーザーの移動後の自転車の現在地と定位置との距離を短くする
# distance_objective = solver.Sum(distances[b][j] * x[b][j] for b in range(B.shape[0]) for j in range(J.shape[0]))
distance_objective = solver.Sum(distances[b][j] * x[b][j] for b in range(df_bikes.shape[0]) for j in range(df_requests.shape[0]))
# 第二項: より多くのユーザーに自転車を割り当てる
# sum_x = solver.Sum(x[b][j] for b in range(B.shape[0]) for j in range(J.shape[0]))
sum_x = solver.Sum(x[b][j] for b in range(df_bikes.shape[0]) for j in range(df_requests.shape[0]))

objective = distance_objective - alpha * sum_x
solver.Minimize(objective)

# 制約条件の定義

# # 各ユーザーは1台の自転車にしか割り当てられない
# for b in range(B.shape[0]):
#     solver.Add(solver.Sum(x[b][j] for j in range(J.shape[0])) <= 1)

# # 各自転車は１人のユーザーにしか割り当てられない
# for j in range(J.shape[0]):
#     solver.Add(solver.Sum(x[b][j] for b in range(B.shape[0])) <= 1)

# 各ユーザーは1台の自転車にしか割り当てられない
for b in range(df_bikes.shape[0]):
    solver.Add(solver.Sum(x[b][j] for j in range(df_requests.shape[0])) <= 1)

# 各自転車は１人のユーザーにしか割り当てられない
for j in range(df_requests.shape[0]):
    solver.Add(solver.Sum(x[b][j] for b in range(df_bikes.shape[0])) <= 1)

# 徒歩30分で移動できる距離
R = 250
# # 半径r内に存在する自転車しかユーザーに割り当てない制約
# for b in range(B.shape[0]):
#     for j in range(J.shape[0]):
#         if initial_distances[b][j] > R:
#             solver.Add(x[b][j] == 0)

# # 他ユーザーに割り当てられていない利用可能な自転車のみを割り当てる
# for b in range(available_bikes.shape[0]):
#     if available_bikes[b] == 0:
#         for j in range(J.shape[0]):
#             solver.Add(x[b][j] == 0)

# 半径r内に存在する自転車しかユーザーに割り当てない制約
for b in range(df_bikes.shape[0]):
    for j in range(df_requests.shape[0]):
        if initial_distances[b][j] > R:
            solver.Add(x[b][j] == 0)

# 他ユーザーに割り当てられていない利用可能な自転車のみを割り当てる
for b in range(available_bikes.shape[0]):
    if available_bikes[b] == 0:
        for j in range(df_requests.shape[0]):
            solver.Add(x[b][j] == 0)

# ソルバーを実行
status = solver.Solve()
print(status)

if status == pywraplp.Solver.OPTIMAL:
    print('解が見つかりました:')
    bike_assignment = []
    for b in range(B.shape[0]):
        for j in range(J.shape[0]):
            if x[b][j].solution_value() == 1:
                bike_assignment.append((b, j))
                print(f"利用者 {j}: 自転車 {b}")
else:
    raise RuntimeError("No feasible solution was found.")

[[x[0,0], x[0,1], x[0,2]], [x[1,0], x[1,1], x[1,2]], [x[2,0], x[2,1], x[2,2]], [x[3,0], x[3,1], x[3,2]], [x[4,0], x[4,1], x[4,2]]]
0
解が見つかりました:
利用者 2: 自転車 3
利用者 0: 自転車 4


In [ ]:
print(bike_assignment)

[(3, 2), (4, 0)]


In [ ]:
import folium
import numpy as np

'''自転車とユーザーを，割り当てられた自転車ごとに異なる色で塗り分けてプロットする関数'''
def plot_result(
    bike_assignment: list[tuple[int, int]],
    user_locations: np.ndarray,
    bike_locations: np.ndarray,
    latitude_range: tuple[float, float],  # 描画範囲 (緯度)
    longitude_range: tuple[float, float],  # 描画範囲 (経度)
):
    # マップを用意
    tiles = "Cartodb Positron"
    m = folium.Map(
        [sum(latitude_range) / 2, sum(longitude_range) / 2],
        tiles=tiles,
        zoom_start=11,
    )

    # ピンクと黄色の色を割り当てる
    colors = ['blue', 'orange']

    # 自転車が割り当てられているかを確認するためのフラグ
    assigned_bikes = {bike_index for bike_index, _ in bike_assignment}
    assigned_users = {user_index for _, user_index in bike_assignment}

    # 割り当てられた自転車のプロット
    for idx, (bike_index, user_index) in enumerate(bike_assignment):
        # 割り当てられた色を選択
        color = colors[idx % 2]  # 2色を交互に使用

        # 自転車の位置をプロット
        bike_latitude, bike_longitude = bike_locations[bike_index]
        folium.Marker(
            location=(bike_latitude, bike_longitude),
            popup=f"bike {bike_index}",
            icon=folium.Icon(
                icon="bicycle", prefix="fa", color="white", icon_color=color
            ),
        ).add_to(m)

        # ユーザーの位置を円でプロット
        user_latitude, user_longitude = user_locations[user_index]

        # ユーザーマーカーを追加
        folium.Marker(
            location=(user_latitude, user_longitude),
            popup=f"User {user_index} assigned to bike {bike_index}",
            icon=folium.Icon(
                icon="user",
                prefix="fa",
                color="white",
                icon_color=color,
            ),
        ).add_to(m)

    # 割り当てられていない自転車のプロット
    for bike_index, (latitude, longitude) in enumerate(bike_locations):
        if bike_index not in assigned_bikes:
            # 割り当てられていない自転車をライトグレーで表示
            folium.Marker(
                location=(latitude, longitude),
                popup="Unassigned bike",
                icon=folium.Icon(
                    icon="bicycle", prefix="fa", color="white", icon_color='lightgray'
                ),
            ).add_to(m)

    # 割り当てられていないユーザーのプロット
    for user_index, (latitude, longitude) in enumerate(user_locations):
        if user_index not in assigned_users:
            # ピンクで表示された未割り当てユーザー
            folium.Marker(
                location=(latitude, longitude),
                popup="Unassigned user",
                icon=folium.Icon(
                    icon="user", prefix="fa", color="white", icon_color='lightgray'
                ),
            ).add_to(m)

    return m

In [ ]:
plot_result(bike_assignment, request_origins, current_locations, latitude_range, longitude_range)

In [ ]:
# 割り当てプロット後に自転車のステータスを更新する
for b, j in bike_assignment:
    # jのtpep_dropoff_datetimeを取得するし自転車ステータス更新する
    B.at[b, 'DODatetime'] = J.loc[j, 'tpep_dropoff_datetime']
    # jのDOLocationIDを取得して自転車のCurrent Locationを更新する
    B.at[b, 'Current Location'] = J.loc[j, 'DOLocationID']

B

,Home Position,Current Location,DODatetime
Bike ID,,,
0,"(40.75302540139779, -73.96722485932337)","(40.75537502442001, -73.97718971701514)",NaT
1,"(40.74630172960795, -73.99076945733133)","(40.75209495412983, -73.975436356298)",NaT
2,"(40.75046184888543, -73.97346615591313)","(40.75046184888543, -73.97346615591313)",NaT
3,"(40.754672378913824, -73.98080231506587)","40.75266371250171,-73.97499968784228",2023-01-01 00:17:36
4,"(40.75701407362949, -73.97694085905479)","40.76029373426692,-73.9699974138823",2023-01-01 00:24:04


In [ ]:
def calculate_total_distance(df_bikes: pd.DataFrame) -> float:
    total_distance = 0.0
    for _, row in df_bikes.iterrows():
        home_position = row['Home Position']
        current_location = row['Current Location']
        total_distance += geodesic(home_position, current_location).m
    return total_distance

# 自転車の再配置コストを計算する
rebalance_cost = calculate_total_distance(B)
print(f"Rebalance Cost: {rebalance_cost} meters")

Rebalance Cost: 3555.581756135627 meters


# 仮想データ
データフレームいじってます。

In [ ]:
bike_data = [
    {'Bike ID': 0, 'Home Position': (40.75302540139779, -73.96722485932337), 'Current Location': (40.75266371250171, -73.97499968784228), 'DODatetime': pd.NaT},
    {'Bike ID': 1, 'Home Position': (40.74630172960795, -73.99076945733133), 'Current Location': (40.75209495412983, -73.975436356298), 'DODatetime': pd.NaT},
    {'Bike ID': 2, 'Home Position': (40.75046184888543, -73.97346615591313), 'Current Location': (40.75046184888543, -73.97346615591313), 'DODatetime': pd.NaT},
    {'Bike ID': 3, 'Home Position': (40.754672378913824, -73.98080231506587), 'Current Location': (40.754672378913824, -73.98080231506587), 'DODatetime': '2023-01-01 00:17:36'},
    {'Bike ID': 4, 'Home Position': (40.75701407362949, -73.97694085905479), 'Current Location': (40.76029373426692, -73.9699974138823), 'DODatetime': '2023-01-01 00:24:04'}
]

    # データフレームに変換
df_bikes = pd.DataFrame(bike_data)
df_bikes.set_index('Bike ID', inplace=True)
df_bikes

,Home Position,Current Location,DODatetime
Bike ID,,,
0,"(40.75302540139779, -73.96722485932337)","(40.75266371250171, -73.97499968784228)",NaT
1,"(40.74630172960795, -73.99076945733133)","(40.75209495412983, -73.975436356298)",NaT
2,"(40.75046184888543, -73.97346615591313)","(40.75046184888543, -73.97346615591313)",NaT
3,"(40.754672378913824, -73.98080231506587)","(40.754672378913824, -73.98080231506587)",2023-01-01 00:17:36
4,"(40.75701407362949, -73.97694085905479)","(40.76029373426692, -73.9699974138823)",2023-01-01 00:24:04


In [ ]:
# 自転車の再配置コストを計算する
rebalance_cost = calculate_total_distance(B)
print(f"Rebalance Cost: {rebalance_cost} meters")

rebalance_cost = calculate_total_distance(df_bikes)
print(f"Rebalance Cost test: {rebalance_cost} meters")

Rebalance Cost: 3555.581756135627 meters
Rebalance Cost test: 2793.9577981395064 meters


# プレゼングラフ作成用

In [ ]:
import folium
import numpy as np

'''ユーザーの位置と自転車の位置をプロットする関数'''
def plot_users_and_bikes(
    user_locations: np.ndarray,  # ユーザーの位置（ピックアップ地点）
    dropoff_locations: np.ndarray,  # ユーザーの位置（ドロップオフ地点）
    bike_locations: np.ndarray,  # 自転車の現在位置
    home_locations: np.ndarray,  # 自転車のホーム位置
    latitude_range: tuple[float, float],  # 描画範囲 (緯度)
    longitude_range: tuple[float, float],  # 描画範囲 (経度)
):
    tiles = "Cartodb Positron"
    # マップの初期化
    m = folium.Map(
        [sum(latitude_range) / 2, sum(longitude_range) / 2],
        tiles=tiles,
        zoom_start=11,
    )

    # ユーザーの位置（ピックアップ地点）をプロット
    for i, (latitude, longitude) in enumerate(user_locations):
        folium.Marker(
            location=(latitude, longitude),
            icon=folium.Icon(icon="user", prefix="fa", color="orange"),
            tooltip=f"Pickup Location {i}: {latitude:.6f}, {longitude:.6f}"
        ).add_to(m)

        # ピックアップ地点に円を描画
        # folium.Circle(
        #     location=(latitude, longitude),
        #     radius=250,
        #     color='orange',
        #     fill=True,
        #     fill_opacity=0.2,
        #     tooltip=f"Pickup Location: {latitude:.6f}, {longitude:.6f}"
        # ).add_to(m)

    # 自転車の現在位置とホーム位置をプロットし、点線で結ぶ
    for i, ((bike_lat, bike_lon), (home_lat, home_lon)) in enumerate(zip(bike_locations, home_locations)):
        # 自転車の現在位置をプロット
        folium.Marker(
            location=(bike_lat, bike_lon),
            icon=folium.Icon(icon="bicycle", prefix="fa", color="green"),
            tooltip=f"Bike {i} Current Location: {bike_lat:.6f}, {bike_lon:.6f}"
        ).add_to(m)

        # 自転車のホーム位置をプロット
        # folium.Marker(
        #     location=(home_lat, home_lon),
        #     icon=folium.Icon(icon="home", prefix="fa", color="green"),
        #     tooltip=f"Bike {i} Home Location: {home_lat:.6f}, {home_lon:.6f}"
        # ).add_to(m)

        # 現在位置とホーム位置を点線で結ぶ
        # folium.PolyLine(
        #     locations=[(home_lat, home_lon), (bike_lat, bike_lon)],
        #     color='gray',
        #     weight=1.5,
        #     opacity=0.5,
        #     dash_array='5, 10',  # 点線を描画
        #     tooltip=f"Path from Home to Current: Bike {i}"
        # ).add_to(m)

    # ピックアップ地点からドロップオフ地点までの黒い矢印を描画
    # for (pu_lat, pu_lon), (do_lat, do_lon) in zip(user_locations, dropoff_locations):
    #     folium.PolyLine(
    #         locations=[(pu_lat, pu_lon), (do_lat, do_lon)],
    #         color='black',
    #         weight=2.5,
    #         opacity=0.7,
    #         tooltip=f"Route from ({pu_lat:.6f}, {pu_lon:.6f}) to ({do_lat:.6f}, {do_lon:.6f})"
    #     ).add_to(m)

    return m


# 地図の描画
plot_users_and_bikes(request_origins, request_destinations, current_locations, home_locations, latitude_range, longitude_range)

In [ ]:
import folium
import numpy as np

'''ユーザーの位置と自転車の位置をプロットする関数'''
def plot_users_and_bikes(
    user_locations: np.ndarray,  # ユーザーの位置（ピックアップ地点）
    dropoff_locations: np.ndarray,  # ユーザーの位置（ドロップオフ地点）
    bike_locations: np.ndarray,  # 自転車の現在位置
    home_locations: np.ndarray,  # 自転車のホーム位置
    latitude_range: tuple[float, float],  # 描画範囲 (緯度)
    longitude_range: tuple[float, float],  # 描画範囲 (経度)
):
    tiles = "Cartodb Positron"
    # マップの初期化
    m = folium.Map(
        [sum(latitude_range) / 2, sum(longitude_range) / 2],
        tiles=tiles,
        zoom_start=11,
    )

    # ユーザーの位置（ピックアップ地点）をプロット
    for i, (latitude, longitude) in enumerate(user_locations):
        folium.Marker(
            location=(latitude, longitude),
            icon=folium.Icon(icon="user", prefix="fa", color="orange"),
            tooltip=f"Pickup Location {i}: {latitude:.6f}, {longitude:.6f}"
        ).add_to(m)

        # ピックアップ地点に円を描画
        folium.Circle(
            location=(latitude, longitude),
            radius=250,
            color='orange',
            fill=True,
            fill_opacity=0.2,
            tooltip=f"Pickup Location: {latitude:.6f}, {longitude:.6f}"
        ).add_to(m)

    # 自転車の現在位置とホーム位置をプロットし、点線で結ぶ
    for i, ((bike_lat, bike_lon), (home_lat, home_lon)) in enumerate(zip(bike_locations, home_locations)):
        # 自転車の現在位置をプロット
        # folium.Marker(
        #     location=(bike_lat, bike_lon),
        #     icon=folium.Icon(icon="bicycle", prefix="fa", color="green"),
        #     tooltip=f"Bike {i} Current Location: {bike_lat:.6f}, {bike_lon:.6f}"
        # ).add_to(m)

        # iが1または2の場合は自転車をグレーアウトしてcontinue
        # 上記のコードはコメントアウトして使う
        if i in [1, 2]:
            # 自転車の現在位置をプロット
            folium.Marker(
                location=(bike_lat, bike_lon),
                icon=folium.Icon(icon="bicycle", prefix="fa", color="lightgray"),
                tooltip=f"Bike {i} Current Location: {bike_lat:.6f}, {bike_lon:.6f}"
            ).add_to(m)
            continue
        else:
            # 自転車の現在位置をプロット
            folium.Marker(
                location=(bike_lat, bike_lon),
                icon=folium.Icon(icon="bicycle", prefix="fa", color="green"),
                tooltip=f"Bike {i} Current Location: {bike_lat:.6f}, {bike_lon:.6f}"
            ).add_to(m)

            # # 自転車のホーム位置をプロット
            # folium.Marker(
            #     location=(home_lat, home_lon),
            #     icon=folium.Icon(icon="home", prefix="fa", color="green"),
            #     tooltip=f"Bike {i} Home Location: {home_lat:.6f}, {home_lon:.6f}"
            # ).add_to(m)

            # # 現在位置とホーム位置を点線で結ぶ
            # folium.PolyLine(
            #     locations=[(home_lat, home_lon), (bike_lat, bike_lon)],
            #     color='black',
            #     weight=2.5,
            #     opacity=0.5,
            #     dash_array='5, 10',  # 点線を描画
            #     tooltip=f"Path from Home to Current: Bike {i}"
            # ).add_to(m)

    # ピックアップ地点からドロップオフ地点までの黒い矢印を描画
    # for (pu_lat, pu_lon), (do_lat, do_lon) in zip(user_locations, dropoff_locations):
    #     folium.PolyLine(
    #         locations=[(pu_lat, pu_lon), (do_lat, do_lon)],
    #         color='black',
    #         weight=2.5,
    #         opacity=0.7,
    #         tooltip=f"Route from ({pu_lat:.6f}, {pu_lon:.6f}) to ({do_lat:.6f}, {do_lon:.6f})"
    #     ).add_to(m)

    return m


# 地図の描画
plot_users_and_bikes(request_origins, request_destinations, current_locations, home_locations, latitude_range, longitude_range)

In [ ]:
import folium
import numpy as np

'''ユーザーの位置と自転車の位置をプロットする関数'''
def plot_users_and_bikes(
    user_locations: np.ndarray,  # ユーザーの位置（ピックアップ地点）
    dropoff_locations: np.ndarray,  # ユーザーの位置（ドロップオフ地点）
    bike_locations: np.ndarray,  # 自転車の現在位置
    home_locations: np.ndarray,  # 自転車のホーム位置
    latitude_range: tuple[float, float],  # 描画範囲 (緯度)
    longitude_range: tuple[float, float],  # 描画範囲 (経度)
):
    tiles = "Cartodb Positron"
    # マップの初期化
    m = folium.Map(
        [sum(latitude_range) / 2, sum(longitude_range) / 2],
        tiles=tiles,
        zoom_start=11,
    )

    # ユーザーの位置（ピックアップ地点）をプロット
    for i, (latitude, longitude) in enumerate(user_locations):
        folium.Marker(
            location=(latitude, longitude),
            icon=folium.Icon(icon="user", prefix="fa", color="orange"),
            tooltip=f"Pickup Location {i}: {latitude:.6f}, {longitude:.6f}"
        ).add_to(m)

        # ピックアップ地点に円を描画
        folium.Circle(
            location=(latitude, longitude),
            radius=250,
            color='orange',
            fill=True,
            fill_opacity=0.2,
            tooltip=f"Pickup Location: {latitude:.6f}, {longitude:.6f}"
        ).add_to(m)

    # 自転車の現在位置とホーム位置をプロットし、点線で結ぶ
    for i, ((bike_lat, bike_lon), (home_lat, home_lon)) in enumerate(zip(bike_locations, home_locations)):
        # 自転車の現在位置をプロット
        # folium.Marker(
        #     location=(bike_lat, bike_lon),
        #     icon=folium.Icon(icon="bicycle", prefix="fa", color="green"),
        #     tooltip=f"Bike {i} Current Location: {bike_lat:.6f}, {bike_lon:.6f}"
        # ).add_to(m)

        # iが1または2の場合は自転車をグレーアウトしてcontinue
        # 上記のコードはコメントアウトして使う
        if i in [1, 2]:
            # 自転車の現在位置をプロット
            folium.Marker(
                location=(bike_lat, bike_lon),
                icon=folium.Icon(icon="bicycle", prefix="fa", color="lightgray"),
                tooltip=f"Bike {i} Current Location: {bike_lat:.6f}, {bike_lon:.6f}"
            ).add_to(m)
            continue
        else:
            # 自転車の現在位置をプロット
            folium.Marker(
                location=(bike_lat, bike_lon),
                icon=folium.Icon(icon="bicycle", prefix="fa", color="green"),
                tooltip=f"Bike {i} Current Location: {bike_lat:.6f}, {bike_lon:.6f}"
            ).add_to(m)

            # 自転車のホーム位置をプロット
            folium.Marker(
                location=(home_lat, home_lon),
                icon=folium.Icon(icon="home", prefix="fa", color="green"),
                tooltip=f"Bike {i} Home Location: {home_lat:.6f}, {home_lon:.6f}"
            ).add_to(m)

            # 現在位置とホーム位置を点線で結ぶ
            folium.PolyLine(
                locations=[(home_lat, home_lon), (bike_lat, bike_lon)],
                color='black',
                weight=2.5,
                opacity=0.5,
                dash_array='5, 10',  # 点線を描画
                tooltip=f"Path from Home to Current: Bike {i}"
            ).add_to(m)

    # ピックアップ地点からドロップオフ地点までの黒い矢印を描画
    for (pu_lat, pu_lon), (do_lat, do_lon) in zip(user_locations, dropoff_locations):
        folium.PolyLine(
            locations=[(pu_lat, pu_lon), (do_lat, do_lon)],
            color='black',
            weight=2.5,
            opacity=0.7,
            tooltip=f"Route from ({pu_lat:.6f}, {pu_lon:.6f}) to ({do_lat:.6f}, {do_lon:.6f})"
        ).add_to(m)

    return m


# 地図の描画
plot_users_and_bikes(request_origins, request_destinations, current_locations, home_locations, latitude_range, longitude_range)

In [ ]:
import folium
import numpy as np

'''自転車とユーザーを，割り当てられた自転車ごとに異なる色で塗り分けてプロットする関数'''
def plot_result(
    bike_assignment: list[tuple[int, int]],
    user_locations: np.ndarray,
    bike_locations: np.ndarray,
    latitude_range: tuple[float, float],  # 描画範囲 (緯度)
    longitude_range: tuple[float, float],  # 描画範囲 (経度)
):
    # マップを用意
    tiles = "Cartodb Positron"
    m = folium.Map(
        [sum(latitude_range) / 2, sum(longitude_range) / 2],
        tiles=tiles,
        zoom_start=11,
    )

    # ピンクと黄色の色を割り当てる
    colors = ['blue', 'orange']

    # 自転車が割り当てられているかを確認するためのフラグ
    assigned_bikes = {bike_index for bike_index, _ in bike_assignment}
    assigned_users = {user_index for _, user_index in bike_assignment}

    # 割り当てられた自転車のプロット
    for idx, (bike_index, user_index) in enumerate(bike_assignment):
        # 割り当てられた色を選択
        color = colors[idx % 2]  # 2色を交互に使用

        # 自転車の位置をプロット
        bike_latitude, bike_longitude = bike_locations[bike_index]
        folium.Marker(
            location=(bike_latitude, bike_longitude),
            popup=f"bike {bike_index}",
            icon=folium.Icon(
                icon="bicycle", prefix="fa", color="white", icon_color=color
            ),
        ).add_to(m)

        # ユーザーの位置を円でプロット
        user_latitude, user_longitude = user_locations[user_index]

        # ユーザーマーカーを追加
        folium.Marker(
            location=(user_latitude, user_longitude),
            popup=f"User {user_index} assigned to bike {bike_index}",
            icon=folium.Icon(
                icon="user",
                prefix="fa",
                color="white",
                icon_color=color,
            ),
        ).add_to(m)

    # 割り当てられていない自転車のプロット
    for bike_index, (latitude, longitude) in enumerate(bike_locations):
        if bike_index not in assigned_bikes:
            # 割り当てられていない自転車をライトグレーで表示
            folium.Marker(
                location=(latitude, longitude),
                popup="Unassigned bike",
                icon=folium.Icon(
                    icon="bicycle", prefix="fa", color="white", icon_color='gray'
                ),
            ).add_to(m)

    # 割り当てられていないユーザーのプロット
    for user_index, (latitude, longitude) in enumerate(user_locations):
        if user_index not in assigned_users:
            # ピンクで表示された未割り当てユーザー
            folium.Marker(
                location=(latitude, longitude),
                popup="Unassigned user",
                icon=folium.Icon(
                    icon="user", prefix="fa", color="white", icon_color='gray'
                ),
            ).add_to(m)

    return m

bike_assignment = [(0, 2), (4, 0)]
plot_result(bike_assignment, request_origins, current_locations, latitude_range, longitude_range)